# RNA-Seq Analysis using Snakemake and Google Cloud Life Sciences API

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a prokaryotic data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression. This tutorial uses a popular workflow manager called ['snakemake'](https://snakemake.readthedocs.io/en/stable/) run via the [Google Cloud Life Sciences API](https://cloud.google.com/life-sciences/docs/reference/rest).

### Step 1: Create a new GS Bucket to store input and output files
Note that your bucket has to be globally unique, so make sure you don't just copy the example here or it won't work

In [8]:
#change this bucket name
%env BUCKET=gls-api-snakemakev2

env: BUCKET=gls-api-snakemakev2


In [9]:
#will only create the bucket if it doesn't yet exist
!gsutil ls gs://$BUCKET >& /dev/null || gsutil mb gs://$BUCKET

In [10]:
#set versioning on the bucket so it can overwrite old files
!gsutil versioning set on gs://$BUCKET

Enabling versioning for gs://gls-api-snakemakev2/...


### STEP 2: Install mambaforge and snakemake
First install mambaforge, then use mamba to install snakemake.

In [ ]:
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda snakemake

### STEP 3: Copy FASTQ Files
In order for this tutorial to run quickly, we will only analyze 50,000 reads from a sample from both sample groups instead of analyzing all the reads from all six samples. These files have been posted on a Google Storage Bucket that we made publicly accessible.

In [11]:
!gsutil -m cp -r gs://me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/ gs://$BUCKET/data/raw_fastq

Copying gs://me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349122_1.fastq [Content-Type=chemical/x-fastq]...
Copying gs://me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349123_1.fastq [Content-Type=application/octet-stream]...
Copying gs://me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349128_1.fastq [Content-Type=chemical/x-fastq]...
Copying gs://me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349122_2.fastq [Content-Type=chemical/x-fastq]...
Copying gs://me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349123_2.fastq [Content-Type=application/octet-stream]...
Copying gs://me-inbre-rnaseq-pipelinev2/data/raw_fastqSub/SRR13349128_2.fastq [Content-Type=chemical/x-fastq]...
/ [6/6 files][ 47.0 MiB/ 47.0 MiB] 100% Done                                    
Operation completed over 6 objects/47.0 MiB.                                     


Create a fake path to data/fastqc so that snakemake can write files to that bucket path, otherwise the pipeline crashes.

In [12]:
!touch blank.txt
!gsutil cp blank.txt gs://$BUCKET/data/fastqc/

Copying file://blank.txt [Content-Type=text/plain]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              


### STEP 4: Copy reference files that will be used by Salmon
Salmon is a tool that aligns RNA-Seq reads to a set of transcripts rather than the entire genome.

In [13]:
!gsutil -m cp -r gs://me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_transcripts.fasta gs://$BUCKET/data/reference/M_chelonae_transcripts.fasta
!gsutil -m cp -r gs://me-inbre-rnaseq-pipelinev2/data/reference/decoys.txt gs://$BUCKET/data/reference/decoys.txt


Copying gs://me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_transcripts.fasta [Content-Type=biosequence/fasta]...
/ [1/1 files][  9.4 MiB/  9.4 MiB] 100% Done                                    
Operation completed over 1 objects/9.4 MiB.                                      
Copying gs://me-inbre-rnaseq-pipelinev2/data/reference/decoys.txt [Content-Type=text/plain]...
/ [1/1 files][   14.0 B/   14.0 B] 100% Done                                    
Operation completed over 1 objects/14.0 B.                                       


### STEP 5: Copy data file for Trimmomatic

In [14]:
!gsutil -m cp gs://me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa gs://$BUCKET/TruSeq3-PE.fa

Copying gs://me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa [Content-Type=application/octet-stream]...
/ [1/1 files][   95.0 B/   95.0 B] 100% Done                                    
Operation completed over 1 objects/95.0 B.                                       


### STEP 6: Copy data and config files that will be used in our snakemake environment

Next download config files for our snakemake environment.

In [15]:
!gsutil -m cp -r gs://me-inbre-rnaseq-pipelinev2/envs/ .
!gsutil -m cp gs://me-inbre-rnaseq-pipelinev2/config.yaml .
!gsutil -m cp gs://me-inbre-rnaseq-pipelinev2/snakefile_ls_api .

Copying gs://me-inbre-rnaseq-pipelinev2/envs/bwa.yaml...
Copying gs://me-inbre-rnaseq-pipelinev2/envs/fastqc.yaml...                     
Copying gs://me-inbre-rnaseq-pipelinev2/envs/fastqc_old.yaml...                 
Copying gs://me-inbre-rnaseq-pipelinev2/envs/multiqc.yaml...                    
Copying gs://me-inbre-rnaseq-pipelinev2/envs/salmon.yaml...                     
Copying gs://me-inbre-rnaseq-pipelinev2/envs/samtools.yaml...                   
Copying gs://me-inbre-rnaseq-pipelinev2/envs/sra-tools.yaml...                  
Copying gs://me-inbre-rnaseq-pipelinev2/envs/trimmomatic.yaml...                
Copying gs://me-inbre-rnaseq-pipelinev2/envs/trinity.yaml...                    
/ [9/9 files][  881.0 B/  881.0 B] 100% Done                                    
Operation completed over 9 objects/881.0 B.                                      
Copying gs://me-inbre-rnaseq-pipelinev2/config.yaml...
/ [1/1 files][   67.0 B/   67.0 B] 100% Done                                 

Add the bucket path to the end of your config file. Since this file was written for running snakemake locally we have to make a few edits to run on the LS API.

In [16]:
!echo 'bucket:' >> config.yaml

In [17]:
!echo '   '$BUCKET >>config.yaml

Add bucket path to the snakefile

In [18]:
!sed -i 's/print(SAMPLES)/BUCKET=config["bucket"]/' snakefile_ls_api

### Step 7: Set up your local environment
You need to generate a [service account key](https://cloud.google.com/iam/docs/creating-managing-service-account-keys) for the compute engine default service account to interact with the Life Sciences API using Snakemake. Download the key and copy it to this VM. Then assign the path of the json file to an environment variable.

In [19]:
%env GOOGLE_APPLICATION_CREDENTIALS=cloud_creds.json

env: GOOGLE_APPLICATION_CREDENTIALS=cloud_creds.json


In [20]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'cloud_creds.json'

Set your project

In [21]:
!gcloud config set project cit-oconnellka-1212

Updated property [core/project].


Initialize a local git repo

In [22]:
!git init

Reinitialized existing Git repository in /home/jupyter/ls_api_snakemake/.git/


Configure conda

In [23]:
conda config --set channel_priority strict


Note: you may need to restart the kernel to use updated packages.


### STEP 8: Run snakemake using the Life Sciences API

Aside from the .yaml config files which information about software, dependencies, and versions -- snakemake uses a snakefile which contains information about a workflow.

This can be a powerful tool as it allows one to operate and think in terms of workflows instead of individual steps. You should open the snakefile to look at it further. It is composed of 'rules' we have created. Snakefiles work largely based on inputs. For a given input/output, there is an associated 'rule' that runs. Snakefiles may take a while to get the idea of what's going on, but in simplest terms here we take an input of .fastq files, and based on the snakefile rules we created, those fastq files are run through the entire workflow. The rule_all at the top determines which rules are run based on the input files for rule_all (which are outputs from the target rules. Comment out rules you don't want to run. 

Snakemake requires that you have a service account key to authenticate with the Life Sciences API. This actually is not necessary to use the API from within a notebook, but Snakemake does require it since Snakemake is expecting you to run the command from your own terminal using the SDK. To see all the commands you can run with Snakemake via the Life Sciences API, check out the [docs](https://snakemake.readthedocs.io/en/stable/executor_tutorial/google_lifesciences.html).

Now we can run the Life Sciences APi. You will see that each rule is submitted as a separate job. If the pipeline crashes, the way to troubleshoot is by reading the API logs, or the snakemake rule logs (same info). You can find the Life Sciences API logs by pasting in the gcloud command given in yellow.

For example: 
```
gcloud beta lifesciences operations describe <JOB_ID>
```
Or you can view the logs by finding the path given for logs, and then use gsutil to copy that file locally, or go to the bucket and double click the file. You can get the job ID for the output file in the green section of the rule print out.

In [26]:
%%time
! snakemake --forceall --snakefile snakefile_ls_api --google-lifesciences --default-remote-prefix $BUCKET --use-conda --google-lifesciences-region us-central1 -j 24 --rerun-incomplete --default-resources "machine_type=n2-standard"

Building DAG of jobs...
Using snakemake/snakemake:v7.8.5 for Google Life Science jobs.
Using shell: /usr/bin/bash
Provided cloud nodes: 24
Job stats:
job                   count    min threads    max threads
------------------  -------  -------------  -------------
all                       1              1              1
fastqc_trimmed            1             12             12
multiqc_trimmed           1              1              1
salmon_index              1              2              2
salmon_quant_reads        2              2              2
trimmomatic_pe_fq         2             12             12
total                     8              1             12

Select jobs to execute...

[Wed Jul 20 20:51:57 2022]
rule salmon_index:
    input: gls-api-snakemakev2/data/reference/M_chelonae_transcripts.fasta, gls-api-snakemakev2/data/reference/decoys.txt
    output: gls-api-snakemakev2/data/reference/transcriptome_index/complete_ref_lens.bin, gls-api-snakemakev2/data/reference/transcr

### STEP 9: Report the top 10 most highly expressed genes in the samples.

Top 10 most highly expressed genes in the wild-type sample. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`


In [ ]:
!gsutil rm gs://$BUCKET/data/quants/SRR13349122_quant
!gsutil rm gs://$BUCKET/data/quants/SRR13349128_quant

In [ ]:
!gsutil ls gs://$BUCKET/data/quants/

In [ ]:
!gsutil cp -r gs://$BUCKET/data/quants/SRR13349122_quant/ .
!gsutil cp -r gs://$BUCKET/data/quants/SRR13349128_quant/ .

In [ ]:
!sort -nrk 5,5 SRR13349122_quant/quant.sf | head -10

Top 10 most highly expressed genes in the double lysogen sample.


In [ ]:
!sort -nrk 5,5 SRR13349128_quant/quant.sf | head -10

### STEP 10: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [ ]:
!grep 'BB28_RS16545' SRR13349122_quant/quant.sf

Use `grep` to report the expression in the double lysogen sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [ ]:
!grep 'BB28_RS16545' SRR13349128_quant/quant.sf